For any doubt in  this code and in Hyperparameter Tuning with keras go to [Keras.tuner](https://keras.io/keras_tuner/) Website

In [20]:
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [21]:
!pip install keras-tuner -q


In [22]:
from tensorflow.keras import layers


In [23]:
import keras_tuner
#build_model(keras_tuner.HyperParameters())

In [25]:
#This is the Code for Hyperparameters 
#Int method
HyperParameters.Int(
    name,
    min_value,
    max_value,
    step=None,
    sampling="linear",
    default=None,
    parent_name=None,
    parent_values=None,
)

NameError: ignored

#Arguments

- name: A string. the name of parameter. Must be unique for each HyperParameter instance in the search space.
- min_value: Float, the lower bound of the range.
- max_value: Float, the upper bound of the range.
- step: Optional float, the distance between two consecutive samples in the range. If left unspecified, it is possible to sample any value in the interval. If sampling="linear", it will be the minimum additve between two samples. If sampling="log", it will be the minimum multiplier between two samples.
- sampling: String. One of "linear", "log", "reverse_log". Defaults to "linear". When sampling value, it always start from a value in range [0.0, 1.0). The sampling argument decides how the value is projected into the range of [min_value, max_value]. "linear": min_value + value * (max_value - min_value) "log": min_value * (max_value / min_value) ^ value "reverse_log": (max_value - min_value * ((max_value / min_value) ^ (1 - value) - 1))
default: Float, the default value to return for the parameter. If unspecified, the default value will be min_value.
- parent_name: Optional string, specifying the name of the parent HyperParameter to use as the condition to activate the current HyperParameter.
- parent_values: Optional list of the values of the parent HyperParameter to use as the condition to activate the current HyperParameter.

Returns:The value of the hyperparameter, or None if the hyperparameter is not active.

In [39]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)): # For this refer above code of Int Method for Hyperparameter Tuning
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log") # for Hyperparameter Documentation https://keras.io/api/keras_tuner/hyperparameters/#hyperparameters
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

#To test if the model builds successfully.
build_model(keras_tuner.HyperParameters())

[Keras.tuner.Randomsearch()](https://keras.io/api/keras_tuner/tuners/random/)

for all kind of Tuner Class[Visis Here](https://keras.io/api/keras_tuner/tuners/)

for Hyperparameter Documentaion  [visit Here](https://keras.io/api/keras_tuner/hyperparameters/#hyperparameters)

### [More Objective We can try ](https://keras.io/api/metrics/)

[keras_tuner.Objective(name, direction)]()

The objective for optimization during tuning.

###Arguments

- name: String. The name of the objective.
- direction: String. The value should be "min" or "max" indicating whether the objective value should be minimized or maximized.


In [40]:
# for this Function see Above Link
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy", # for this function see above code of more objective we can try and objective funtion
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)


In [41]:
tuner.search_space_summary()


Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [29]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))


Trial 3 Complete [00h 00m 35s]
val_accuracy: 0.9570500254631042

Best val_accuracy So Far: 0.9570500254631042
Total elapsed time: 00h 02m 03s


Now, coming to the best part, we will use an awesome function that comes with this tuner, i.e. the ‘tuner_search.get_best_models()’ function, that will report you a summary of the best model to use for your image classification data!

[model=tuner_search.get_best_models(num_models=1)[0]]()


In [30]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 96)                75360     
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 76,330
Trainable params: 76,330
Non-trainable params: 0
_________________________________________________________________


In [31]:
tuner.results_summary()


Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 1
units_0: 96
activation: relu
dropout: False
lr: 0.009021311428169284
units_1: 96
units_2: 320
Score: 0.9570500254631042

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 288
activation: tanh
dropout: True
lr: 0.00022419548935667316
units_1: 160
units_2: 32
Score: 0.9537000060081482

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 448
activation: tanh
dropout: False
lr: 0.0038025483399172595
units_1: 32
Score: 0.9451499879360199


In [32]:
best_model.fit(x_train,y_train, epochs=10, validation_split=0.1, initial_epoch=3)


Epoch 4/10
1407/1407 [==============================] - 6s 3ms/step - loss: 0.1297 - accuracy: 0.9628 - val_loss: 0.1316 - val_accuracy: 0.9620
Epoch 5/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.1217 - accuracy: 0.9667 - val_loss: 0.1975 - val_accuracy: 0.9554
Epoch 6/10
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1051 - accuracy: 0.9713 - val_loss: 0.2405 - val_accuracy: 0.9502
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.1091 - accuracy: 0.9712 - val_loss: 0.1745 - val_accuracy: 0.9610
Epoch 8/10
1407/1407 [==============================] - 5s 3ms/step - loss: 0.1003 - accuracy: 0.9743 - val_loss: 0.2190 - val_accuracy: 0.9578
Epoch 9/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0857 - accuracy: 0.9773 - val_loss: 0.2240 - val_accuracy: 0.9592
Epoch 10/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0947 - accuracy: 0.9762 - val_loss: 0.2219 - val_accuracy

#Retrain the model
If you want to train the model with the entire dataset, you may retrieve the best hyperparameters and retrain the model by yourself.


In [33]:

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)


1875/1875 [==============================] - 6s 3ms/step - loss: 0.2398 - accuracy: 0.9283


HyperModel class is subclassed to define a model building and training process. The build() method of the HyperModel class is used to create a Keras model using the hyperparameters, and the fit() method is used to train the model and return the training history.

In the fit() method, the shuffle argument in model.fit() is tuned. The shuffle argument controls whether the training data is shuffled before each epoch. Shuffling the training data can help to improve the model's generalization performance.

In [34]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )


In [35]:
#Again, we can do a quick check to see if the code works correctly.

hp = keras_tuner.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, np.random.rand(100, 28, 28), np.random.rand(100, 10))

4/4 [==============================] - 1s 5ms/step - loss: 13.1279 - accuracy: 0.1200


#Tune data preprocessing
To tune data preprocessing, we just add an additional step in HyperModel.fit(), where we can access the dataset from the arguments. In the following code, we tune whether to normalize the data before training the model. This time we explicitly put x and y in the function signature because we need to use them.

In [36]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, x, y, **kwargs):
        if hp.Boolean("normalize"):
            x = layers.Normalization()(x)
        return model.fit(
            x,
            y,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )


hp = keras_tuner.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, np.random.rand(100, 28, 28), np.random.rand(100, 10))

4/4 [==============================] - 1s 5ms/step - loss: 14.6753 - accuracy: 0.0600


In [37]:
def build_regressor(hp):
    model = keras.Sequential(
        [
            layers.Dense(units=hp.Int("units", 32, 128, 32), activation="relu"),
            layers.Dense(units=1),
        ]
    )
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        # Objective is one of the metrics.
        metrics=[keras.metrics.MeanAbsoluteError()],
    )
    return model


tuner = keras_tuner.RandomSearch(
    hypermodel=build_regressor,
    # The objective name and direction.
    # Name is the f"val_{snake_case_metric_class_name}".
    objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
    max_trials=3,
    overwrite=True,
    directory="my_dir",
    project_name="built_in_metrics",
)

tuner.search(
    x=np.random.rand(100, 10),
    y=np.random.rand(100, 1),
    validation_data=(np.random.rand(20, 10), np.random.rand(20, 1)),
)

tuner.results_summary()

Trial 3 Complete [00h 00m 02s]
val_mean_absolute_error: 0.2734336853027344

Best val_mean_absolute_error So Far: 0.2341572344303131
Total elapsed time: 00h 00m 05s
Results summary
Results in my_dir/built_in_metrics
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 1 summary
Hyperparameters:
units: 32
Score: 0.2341572344303131

Trial 2 summary
Hyperparameters:
units: 64
Score: 0.2734336853027344

Trial 0 summary
Hyperparameters:
units: 96
Score: 0.356167197227478


Keep Keras code separate
You can keep all your Keras code unchanged and use KerasTuner to tune it. It is useful if you cannot modify the Keras code for some reason.

It also gives you more flexibility. You don't have to separate the model building and training code apart. However, this workflow would not help you save the model or connect with the TensorBoard plugins.

To save the model, you can use trial.trial_id, which is a string to uniquely identify a trial, to construct different paths to save the models from different trials.

In [38]:
import os


def keras_code(units, optimizer, saving_path):
    # Build model
    model = keras.Sequential(
        [layers.Dense(units=units, activation="relu"), layers.Dense(units=1),]
    )
    model.compile(
        optimizer=optimizer, loss="mean_squared_error",
    )

    # Prepare data
    x_train = np.random.rand(100, 10)
    y_train = np.random.rand(100, 1)
    x_val = np.random.rand(20, 10)
    y_val = np.random.rand(20, 1)

    # Train & eval model
    model.fit(x_train, y_train)

    # Save model
    model.save(saving_path)

    # Return a single float as the objective value.
    # You may also return a dictionary
    # of {metric_name: metric_value}.
    y_pred = model.predict(x_val)
    return np.mean(np.abs(y_pred - y_val))


class MyTuner(keras_tuner.RandomSearch):
    def run_trial(self, trial, **kwargs):
        hp = trial.hyperparameters
        return keras_code(
            units=hp.Int("units", 32, 128, 32),
            optimizer=hp.Choice("optimizer", ["adam", "adadelta"]),
            saving_path=os.path.join("/tmp", trial.trial_id),
        )


tuner = MyTuner(
    max_trials=3, overwrite=True, directory="my_dir", project_name="keep_code_separate",
)
tuner.search()
# Retraining the model
best_hp = tuner.get_best_hyperparameters()[0]
keras_code(**best_hp.values, saving_path="/tmp/best_model")

Trial 3 Complete [00h 00m 02s]
default_objective: 1.0694726379974262

Best default_objective So Far: 0.4292792261175326
Total elapsed time: 00h 00m 05s
4/4 [==============================] - 1s 5ms/step - loss: 0.1492


1/1 [==============================] - 0s 39ms/step


0.2695190293193414